# 1 Assignment 3

1. Generate a function named **beta_OLS**. This function must return the estimated beta using OLS. The inputs of this function should be `X`, `y`, and `intercept`. `X` (covariables) must be a **pd.DataFrame** and `y` (endog) must be a **pd.Series** and `intercept` can be `True` or `False`, by default `True`. When `intercept` is `False`, the estimated beta does not include **intercept**. Also, you must specify the type of your function's parameters and output and It must raise an error if the inputs do not meet the requirements. The function's output must be a **pd.DataFrame** (`n`, 1) where `n` is the total number of regressors ( it includes the **Intercept** when `intercept` is `True`). The column should be named as **Coef.** and the row index should be named as the original name of the columns in `X` input. Your results should look like `beta_OLS_output`. Apply your function to find $\widehat{\boldsymbol{\beta}}^{(OLS)}$ of the equation below. Use greene data.  **Hint: Use NumPy to generate the OLS beta and check [this link](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_index.html) to change the index name of a pd.Dataframe, [link](https://notebooks.githubusercontent.com/view/ipynb?browser=chrome&color_mode=auto&commit=69c80e1f2c1c268f0480a32932262201785a576c&device=unknown_device&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f616c6578616e6465727175697370652f4469706c6f6d61646f5f505543502f363963383065316632633163323638663034383061333239333232363232303137383561353736632f4c6563747572655f352f4c6563747572655f352e6970796e62&logged_in=true&nwo=alexanderquispe%2FDiplomado_PUCP&path=Lecture_5%2FLecture_5.ipynb&platform=windows&repository_id=427747212&repository_type=Repository&version=96##5.1.7.).**

$$
\widehat{\boldsymbol{\beta}}^{(OLS)} = \left( \mathbf{X}^\top \mathbf{X}\right)^{-1} \mathbf{X}^\top \mathbf{Y}
$$


$$
\begin{aligned} 
lnCT &= \beta_{0}+\beta_{q}lnq+ \beta_{qq}(lnq)^2+\beta_{q1}lnqlnp_1+\beta_{q2}lnqlnp_2+ \beta_{q3}lnqlnp_{3} +\beta_{1}lnp_1+\beta_{2}lnp_2+ \beta_{3}lnp_3 \\
& + \beta_{11}(lnp_{1})^2+ \beta_{22}(lnp_{2})^2+ \beta_{33}(lnp_{3})^2 + \beta_{12}lnp_{1}lnp_{2}+ \beta_{13}lnp_{1}lnp_{3}+\beta_{23}lnp_{2}lnp_{3} 
\end{aligned}
$$


In [7]:
import pandas as pd
import numpy as np
import pickle
import scipy.stats as stats

In [8]:
greene = pd.read_csv(r"../../_data/christensen_greene_f4.csv")
ct = greene.COST.values
q = greene.Q.values
p1 = greene.PL.values
p2 = greene.PF.values
p3 = greene.PK.values

In [9]:
# Generation of data

y = np.log(  ct  ) # lnCT

c = np.ones( len( y ) ) # Constant
xq = np.log( q ) # lnq
xqq = np.log( q )*np.log( q ) # lnq^2
xq1 = np.log( q )*np.log( p1 ) # lnq*lnp_1
xq2 = np.log( q )*np.log( p2 ) # lnq*lnp_2
xq3 = np.log( q )*np.log( p3 ) # lnq*ln_3
x1 = np.log( p1 ) # lnp_1
x2 = np.log( p2 ) # ln_p2
x3 = np.log( p3 ) # ln_p3
x11 = 0.5*np.log( p1 )*np.log( p1 ) # 0.5ln2p_1
x22 = 0.5*np.log( p2 )*np.log( p2 ) # 0.5ln2p_2
x33 = 0.5*np.log( p3 )*np.log( p3 ) # 0.5ln2p_3
x12 = 0.5*np.log( p1 )*np.log( p2 ) # lnp_1*lnp_2
x13 = 0.5*np.log( p1 )*np.log( p3 ) # lnp_1*lnp_3
x23 = 0.5*np.log( p2 )*np.log( p3 ) # lnp_2*lnp_3

columns = [  "ln_cost", "lnq", "(lnq)^2", "(lnq)(lnp1)", "(lnq)(lnp2)",
          "(lnq)(lnp3)", "lnp1", "lnp2", "lnp3", "(lnp1)^2",
          "(lnp2)^2", "(lnp3)^2", "(lnp1)(lnp2)", "(lnp1)(lnp3)", "(lnp2)(lnp3)" ]

data_val = np.asarray( [ y, xq,  xqq, xq1, xq2,
                xq3, x1,  x2,  x3,  x11, 
                x22, x33, x12, x13, x23 ] ).T

data = pd.DataFrame(  data_val , columns = columns  )

### Saving data

In [10]:
file_to_store = open( r'../../_data/data_greene.pkl' , "wb" )
pickle.dump( data, file_to_store )

### Defining function

In [11]:
def beta_OLS( X : pd.DataFrame , y : pd.Series , intercept = True  ) -> pd.DataFrame:
    
    if not isinstance( X, pd.DataFrame ):
        raise TypeError( "X must be a pd.DataFrame." )
    
    if not isinstance( y , pd.Series ):
        raise TypeError( "y must be a pd.Series." )
        
    if intercept == True:
        
        X[ 'Intercept' ] = 1
        # setting order of columns
        cols = X.columns.tolist()
        new_cols_orders = [ cols[ -1 ] ] + cols[ 0:-1 ]
        X = X.loc[ : , new_cols_orders ]
        
    else:
        pass
        # Data remains the same
    
    # From  values
    X_np = X.values
    y_np = y.values.reshape( -1 , 1 )
    
    # get beta_ols
    beta_ols = np.linalg.inv( X_np.T @ X_np ) @ ( X_np.T @ y_np )
    
    # get columns from X
    index_names = X.columns
    # best look appereance of the output
    beta_OLS_output = pd.DataFrame( beta_ols , index = index_names , columns = [ 'Coef.' ] )
    
    return beta_OLS_output

### Getting filtered data to regressions

In [12]:
# selecting columns
X = data.iloc[ : , 1: ]
y = data.ln_cost

# estimating output
beta_OLS_output = beta_OLS( X , y )

# Save results
file_to_store = open( r'../../_data/beta_OLS_output.pkl' , "wb" )
pickle.dump( beta_OLS_output, file_to_store)

beta_OLS_output

,Coef.
Intercept,-76.259259
lnq,-1.080425
(lnq)^2,0.026489
(lnq)(lnp1),0.131041
(lnq)(lnp2),0.058652
(lnq)(lnp3),0.040144
lnp1,14.718292
lnp2,-0.894733
lnp3,6.380797
(lnp1)^2,-1.538527


2. Generate a new function named **var_OLS**. This function must return the estimated variance for OLS. The inputs of this function should be `X`, `y`, and `intercept`. `X` (covariables) must be a **pd.DataFrame** and `y` (endog) must be a **pd.Series** and `intercept` can be `True` or `False`, by default `True`. When `intercept` is `False`, the estimated variance does not include **intercept** in the `X` regressor. Also, you must specify the type of your function's parameters and output and It must raise an error if the inputs do not meet the requirements. The output of the function should look like `var_OLS_output` where the columns and the index use the name of `X` columns.  Apply your function to find $\mathbb{V}{\rm ar} (\widehat{\boldsymbol{\beta}}^{(OLS)})$ of the equation below. Use greene data.  **Hint: Use numpy, `def` function, and `columns` method, [link](https://notebooks.githubusercontent.com/view/ipynb?browser=chrome&color_mode=auto&commit=69c80e1f2c1c268f0480a32932262201785a576c&device=unknown_device&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f616c6578616e6465727175697370652f4469706c6f6d61646f5f505543502f363963383065316632633163323638663034383061333239333232363232303137383561353736632f4c6563747572655f352f4c6563747572655f352e6970796e62&logged_in=true&nwo=alexanderquispe%2FDiplomado_PUCP&path=Lecture_5%2FLecture_5.ipynb&platform=windows&repository_id=427747212&repository_type=Repository&version=96##5.1.7.).**

$$
\widehat{\boldsymbol{\varepsilon}} = \mathbf{Y} - \mathbf{X} \widehat{\boldsymbol{\beta}}_{OLS}
$$


$$
\widehat{\sigma}^2_{OLS} = \dfrac{\widehat{\boldsymbol{\varepsilon}}^\top \widehat{\boldsymbol{\varepsilon}}}{N - (k+1)}
$$


$$
\begin{aligned}
\mathbb{V}{\rm ar} (\widehat{\boldsymbol{\beta}}^{(OLS)}) = 
\sigma^2 \left( \mathbf{X}^\top  \mathbf{X}\right)^{-1}
\end{aligned}
$$

$$
\begin{aligned} 
lnCT &= \beta_{0}+\beta_{q}lnq+ \beta_{qq}(lnq)^2+\beta_{q1}lnqlnp_1+\beta_{q2}lnqlnp_2+ \beta_{q3}lnqlnp_{3} +\beta_{1}lnp_1+\beta_{2}lnp_2+ \beta_{3}lnp_3 \\
& + \beta_{11}(lnp_{1})^2+ \beta_{22}(lnp_{2})^2+ \beta_{33}(lnp_{3})^2 + \beta_{12}lnp_{1}lnp_{2}+ \beta_{13}lnp_{1}lnp_{3}+\beta_{23}lnp_{2}lnp_{3} 
\end{aligned}
$$


In [13]:
def var_OLS( X : pd.DataFrame , y : pd.Series , intercept = True  ) -> pd.DataFrame:
    
    if not isinstance( X, pd.DataFrame ):
        raise TypeError( "X must be a pd.DataFrame." )
    
    if not isinstance( y , pd.Series ):
        raise TypeError( "y must be a pd.Series." )
        
    if intercept == True:
        
        X[ 'Intercept' ] = 1
        # setting order of columns
        cols = X.columns.tolist()
        new_cols_orders = [ cols[ -1 ] ] + cols[ 0:-1 ]
        X = X.loc[ : , new_cols_orders ]
        
    else:
        pass
        # Data remains the same
    
    # From  values
    X_np = X.values
    y_np = y.values.reshape( -1 , 1 )
    
    # get beta_ols
    beta_ols = beta_OLS( X, y, intercept = False ).values.reshape( -1, 1 )
    
    # get errors
    e = y_np - ( X_np @ beta_ols )
    
    # Getting the error variance
    N = X.shape[ 0 ]
    total_parameters = X.shape[ 1 ]
    error_var = ( (e.T @ e)[ 0 ] )/( N - total_parameters )
    
    # Variance
    var_OLS =  error_var * np.linalg.inv( X_np.T @ X_np )
    
    # get columns from X
    index_names = X.columns
    # best look appereance of the output
    var_OLS_output = pd.DataFrame( var_OLS , index = index_names , columns = index_names )
    
    return var_OLS_output

In [14]:
# estimating output
var_OLS_output = var_OLS( X , y )

# Save results
file_to_store = open( r'../../_data/var_OLS_output.pkl' , "wb" )
pickle.dump( var_OLS_output, file_to_store)

var_OLS_output

,Intercept,lnq,(lnq)^2,(lnq)(lnp1),(lnq)(lnp2),(lnq)(lnp3),lnp1,lnp2,lnp3,(lnp1)^2,(lnp2)^2,(lnp3)^2,(lnp1)(lnp2),(lnp1)(lnp3),(lnp2)(lnp3)
Intercept,1465.361133,0.192537,-0.011698,-0.026677,0.015949,0.039538,-262.932570,-8.392678,-128.192047,23.761713,-0.533889,7.081277,1.980507,21.804881,0.394870
lnq,0.192537,0.141034,0.000308,-0.013459,0.000161,-0.006089,-0.186223,0.148320,-0.090714,0.030641,0.000762,-0.001796,-0.030093,0.035093,-0.007141
(lnq)^2,-0.011698,0.000308,0.000006,-0.000036,-0.000005,-0.000011,0.001491,0.001052,0.000862,-0.000086,0.000026,-0.000120,-0.000198,-0.000028,-0.000097
(lnq)(lnp1),-0.026677,-0.013459,-0.000036,0.001466,-0.000168,0.000348,0.021561,-0.015127,0.005430,-0.003923,0.000038,0.000571,0.003700,-0.002282,-0.000170
(lnq)(lnp2),0.015949,0.000161,-0.000005,-0.000168,0.000653,-0.000189,-0.008293,0.003306,0.008035,0.001175,-0.000340,-0.000205,-0.000340,-0.000289,-0.002751
(lnq)(lnp3),0.039538,-0.006089,-0.000011,0.000348,-0.000189,0.000888,-0.000337,-0.008853,0.000268,0.000421,-0.000066,-0.000134,0.000204,-0.003074,0.004424
lnp1,-262.932569,-0.186223,0.001491,0.021561,-0.008293,-0.000337,50.194722,-1.253531,19.099543,-4.777971,0.140987,-0.840307,0.076832,-3.576317,0.264314
lnp2,-8.392678,0.148320,0.001052,-0.015127,0.003306,-0.008853,-1.253531,8.078742,-0.214871,0.323286,-0.117293,-0.066294,-1.370100,0.398963,-0.734776
lnp3,-128.192047,-0.090714,0.000862,0.005430,0.008035,0.000268,19.099543,-0.214871,20.513714,-1.410427,0.047330,-1.526100,0.072458,-3.111107,-0.145182
(lnp1)^2,23.761713,0.030641,-0.000086,-0.003923,0.001175,0.000421,-4.777971,0.323286,-1.410427,0.479422,-0.012318,0.045414,-0.053651,0.281523,-0.026624


3. Generate a new function named as `reg_OLS` that uses your previos functions `beta_OLS` and `var_OLS` to return an ouput similar to `reg_OLS_output`. The inputs of this function should be `X`, `y`, and `intercept`. `X` (covariables) must be a **pd.DataFrame** and `y` (endog) must be a **pd.Series** and `intercept` can be `True` or `False`, by default `True`. When `intercept` is `False`, the output does not include **intercept**. Also, you must specify the type of your function's parameters and output and It must raise an error if the inputs do not meet the requirements. Apply your function to the equation below. Use greene data.**Hint: Use numpy, `def` function, and `columns` method, [link](https://notebooks.githubusercontent.com/view/ipynb?browser=chrome&color_mode=auto&commit=69c80e1f2c1c268f0480a32932262201785a576c&device=unknown_device&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f616c6578616e6465727175697370652f4469706c6f6d61646f5f505543502f363963383065316632633163323638663034383061333239333232363232303137383561353736632f4c6563747572655f352f4c6563747572655f352e6970796e62&logged_in=true&nwo=alexanderquispe%2FDiplomado_PUCP&path=Lecture_5%2FLecture_5.ipynb&platform=windows&repository_id=427747212&repository_type=Repository&version=96##5.1.7.).**

$$
\begin{aligned} 
lnCT &= \beta_{0}+\beta_{q}lnq+ \beta_{qq}(lnq)^2+\beta_{q1}lnqlnp_1+\beta_{q2}lnqlnp_2+ \beta_{q3}lnqlnp_{3} +\beta_{1}lnp_1+\beta_{2}lnp_2+ \beta_{3}lnp_3 \\
& + \beta_{11}(lnp_{1})^2+ \beta_{22}(lnp_{2})^2+ \beta_{33}(lnp_{3})^2 + \beta_{12}lnp_{1}lnp_{2}+ \beta_{13}lnp_{1}lnp_{3}+\beta_{23}lnp_{2}lnp_{3} 
\end{aligned}
$$


$$
\widehat{\boldsymbol{\varepsilon}} = \mathbf{Y} - \mathbf{X} \widehat{\boldsymbol{\beta}}_{RLS}
$$


$$
\widehat{\sigma}^2_{RLS} = \dfrac{\widehat{\boldsymbol{\varepsilon}}^\top \widehat{\boldsymbol{\varepsilon}}}{N - (k+1 - M)}
$$

In [15]:
def reg_OLS( X : pd.DataFrame , y : pd.Series , intercept = True  ) -> pd.DataFrame:
    
    if not isinstance( X, pd.DataFrame ):
        raise TypeError( "X must be a pd.DataFrame." )
    
    if not isinstance( y , pd.Series ):
        raise TypeError( "y must be a pd.Series." )
    
    # get beta_OLS
    beta_df = beta_OLS( X , y, intercept = intercept )
    index_names = beta_df.index.tolist()
    beta = beta_df.values
    
    # get variance
    var_cov = var_OLS( X , y, intercept = True ).values
    
    # get beta standard errors
    beta_se = np.sqrt( np.diag( var_cov ) )
    
    # Calculate the test statistic for each coefficient:
    t_stat = beta.ravel() / np.sqrt( np.diag( var_cov ) )
    
    # Calculate the associated p-value:
    N = X.shape[ 0 ]
    k = beta.size
    p_val = 2 * stats.t.cdf( x = - np.abs( t_stat ), df =  N - k )
    
    # Confidence interval
    bound = beta_se * stats.t.ppf( ( 1 + 0.95 ) / 2., N - k )
    up_bd = beta.ravel() + bound
    lw_bd = beta.ravel() - bound
    
    table_data ={  'Coef.'    : beta.ravel() ,
                   "Std.Err." : beta_se.ravel(),
                   "t"        : t_stat.ravel(),
                   "P>|t|"    : p_val.ravel(), 
                   "[0.025"   : lw_bd.ravel(),
                   "0.975]"   : up_bd.ravel()
                }
    
    reg_OLS = pd.DataFrame( table_data , index = index_names )
    
    return reg_OLS

In [16]:
reg_OLS_output = reg_OLS( X, y )


file_to_store = open( r'../../_data/reg_OLS_output.pkl' , "wb" )
pickle.dump( reg_OLS_output, file_to_store)

reg_OLS_output

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
Intercept,-76.259259,38.280036,-1.992142,4.825953e-02,-151.927107,-0.591411
lnq,-1.080425,0.375545,-2.876952,4.630885e-03,-1.822762,-0.338088
(lnq)^2,0.026489,0.002357,11.240060,2.112040e-21,0.021831,0.031148
(lnq)(lnp1),0.131041,0.038284,3.422854,8.084865e-04,0.055365,0.206716
(lnq)(lnp2),0.058652,0.025549,2.295650,2.315002e-02,0.008149,0.109155
(lnq)(lnp3),0.040144,0.029791,1.347518,1.799447e-01,-0.018744,0.099033
lnp1,14.718292,7.084823,2.077440,3.954949e-02,0.713777,28.722807
lnp2,-0.894733,2.842313,-0.314790,7.533796e-01,-6.513111,4.723645
lnp3,6.380797,4.529207,1.408811,1.610617e-01,-2.572051,15.333645
(lnp1)^2,-1.538527,0.692403,-2.222011,2.785341e-02,-2.907195,-0.169860


In [17]:
import statsmodels.api as sm

$$
\begin{aligned} 
\beta_{1} + \beta_{2} + \beta_{3} &= 1 \\
\beta_{q1} + \beta_{q2} + \beta_{q3} &= 0 \\
\beta_{11} + \beta_{12} + \beta_{13} &= 0 \\
\beta_{21} + \beta_{22} + \beta_{23} &= 0 \\
\beta_{31} + \beta_{32} + \beta_{33} &= 0 \\
\beta_{ij} = \beta_{ji}
\end{aligned}
$$

In [43]:
new_cols = []
for string1 in X.columns.tolist():
    string1 = string1.replace( "(" , "_")
    string1 = string1.replace( ")" , "_")
    string1 = string1.replace( "^" , "_")
    
    new_cols.append( string1 )

In [39]:
X_1 = X.copy()

In [45]:
X_1.columns = new_cols

In [46]:
new_cols

['lnq',
 '_lnq__2',
 '_lnq__lnp1_',
 '_lnq__lnp2_',
 '_lnq__lnp3_',
 'lnp1',
 'lnp2',
 'lnp3',
 '_lnp1__2',
 '_lnp2__2',
 '_lnp3__2',
 '_lnp1__lnp2_',
 '_lnp1__lnp3_',
 '_lnp2__lnp3_',
 'Intercept']

In [49]:
a = sm.GLM( y, X_1 ).fit_constrained( " lnp1 + lnp2 + lnp3 = 1,  _lnq__lnp1_ + _lnq__lnp2_ + _lnq__lnp3_ = 0, _lnp1__2 + _lnp1__lnp2_ + _lnp1__lnp3_ = 0, _lnp1__lnp2_ + _lnp2__2 + _lnp2__lnp3_ = 0, _lnp1__lnp3_ + _lnp2__lnp3_ + _lnp3__2 = 0 " )

In [51]:
a.summary2().tables[1]

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
lnq,0.461368,0.126550,3.645733,2.666312e-04,0.213334,0.709402
_lnq__2,0.029978,0.002242,13.372045,8.808564e-41,0.025584,0.034372
_lnq__lnp1_,-0.004354,0.024186,-0.180042,8.571199e-01,-0.051757,0.043048
_lnq__lnp2_,0.036778,0.025582,1.437666,1.505288e-01,-0.013361,0.086917
_lnq__lnp3_,-0.032424,0.022641,-1.432096,1.521163e-01,-0.076798,0.011951
lnp1,0.215566,1.073111,0.200880,8.407926e-01,-1.887693,2.318826
lnp2,0.402923,0.585771,0.687851,4.915465e-01,-0.745166,1.551012
lnp3,0.381511,0.561622,0.679302,4.969467e-01,-0.719248,1.482269
_lnp1__2,-0.008712,0.149001,-0.058471,9.533735e-01,-0.300749,0.283325
_lnp2__2,0.005156,0.169478,0.030421,9.757313e-01,-0.327015,0.337326


In [81]:
restricted_OLS_output

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
Intercept,-7.111200,2.755866,-2.580387,1.091364e-02,-12.560384,-1.662016
lnq,0.461368,0.119721,3.853679,1.776124e-04,0.224642,0.698094
(lnq)^2,0.029978,0.002121,14.134763,1.312279e-28,0.025784,0.034171
(lnq)(lnp1),-0.004354,0.022880,-0.190311,8.493452e-01,-0.049596,0.040887
(lnq)(lnp2),0.036778,0.024201,1.519668,1.308823e-01,-0.011075,0.084631
(lnq)(lnp3),-0.032424,0.021419,-1.513780,1.323685e-01,-0.074775,0.009928
lnp1,0.215566,1.015206,0.212338,8.321570e-01,-1.791804,2.222936
lnp2,0.402923,0.554162,0.727085,4.684055e-01,-0.692824,1.498670
lnp3,0.381511,0.531316,0.718048,4.739416e-01,-0.669063,1.432084
(lnp1)^2,-0.008712,0.140961,-0.061806,9.508067e-01,-0.287435,0.270011


In [76]:
cd = restricted_OLS_output.copy()

In [77]:
cd

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
Intercept,-7.111200,3.016756,-2.357234,1.981676e-02,-13.076242,-1.146158
lnq,0.461368,0.131055,3.520412,5.842781e-04,0.202232,0.720504
(lnq)^2,0.029978,0.002322,12.912386,1.664058e-25,0.025387,0.034568
(lnq)(lnp1),-0.004354,0.025047,-0.173853,8.622360e-01,-0.053879,0.045170
(lnq)(lnp2),0.036778,0.026492,1.388247,1.672989e-01,-0.015606,0.089161
(lnq)(lnp3),-0.032424,0.023447,-1.382868,1.689387e-01,-0.078785,0.013937
lnp1,0.215566,1.111312,0.193975,8.464810e-01,-1.981835,2.412968
lnp2,0.402923,0.606623,0.664207,5.076664e-01,-0.796555,1.602401
lnp3,0.381511,0.581614,0.655951,5.129478e-01,-0.768518,1.531539
(lnp1)^2,-0.008712,0.154305,-0.056461,9.550562e-01,-0.313821,0.296396


In [22]:
sm.OLS( y, X ).fit().summary2().tables[1]

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
lnq,-1.080425,0.375545,-2.876952,4.630885e-03,-1.822762,-0.338088
(lnq)^2,0.026489,0.002357,11.240060,2.112040e-21,0.021831,0.031148
(lnq)(lnp1),0.131041,0.038284,3.422854,8.084865e-04,0.055365,0.206716
(lnq)(lnp2),0.058652,0.025549,2.295650,2.315002e-02,0.008149,0.109155
(lnq)(lnp3),0.040144,0.029791,1.347518,1.799447e-01,-0.018744,0.099033
lnp1,14.718293,7.084823,2.077440,3.954949e-02,0.713777,28.722808
lnp2,-0.894733,2.842313,-0.314790,7.533796e-01,-6.513110,4.723645
lnp3,6.380797,4.529207,1.408811,1.610617e-01,-2.572051,15.333645
(lnp1)^2,-1.538527,0.692403,-2.222011,2.785341e-02,-2.907195,-0.169860
(lnp2)^2,-0.073506,0.182039,-0.403791,6.869699e-01,-0.433340,0.286329


4. Generate a new function named as `restricted_OLS` that uses your previos functions `beta_OLS` and `var_OLS` to return an ouput similar to `reg_OLS_output`. The inputs of this function should be `X`, `y`,  `L`, `r`, and `intercept`. `X` (covariables) must be a **pd.DataFrame** and `y` (endog) must be a **pd.Series**,  `L` (coef) must be a **np.array**, `r` must be a **np.array**,  and `intercept` can be `True` or `False`.  When `intercept` is `False`, the estimated beta does not include **intercept**. Also, you must specify the type of your function's parameters and output and It must raise an error if the inputs do not meet the requirements. The ouput of this function should look like `reg_OLS_ouput`. Apply your function to the equation below. Use greene data.
 **Hint: Use numpy, `def` function, and `columns` method, [link](https://notebooks.githubusercontent.com/view/ipynb?browser=chrome&color_mode=auto&commit=69c80e1f2c1c268f0480a32932262201785a576c&device=unknown_device&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f616c6578616e6465727175697370652f4469706c6f6d61646f5f505543502f363963383065316632633163323638663034383061333239333232363232303137383561353736632f4c6563747572655f352f4c6563747572655f352e6970796e62&logged_in=true&nwo=alexanderquispe%2FDiplomado_PUCP&path=Lecture_5%2FLecture_5.ipynb&platform=windows&repository_id=427747212&repository_type=Repository&version=96##5.1.7.).**

$$
\hat{\boldsymbol{\beta}}^{(RLS)} =\hat{\boldsymbol{\beta}}^{(OLS)} - \left( \boldsymbol{X}^\top \boldsymbol{X} \right)^{-1} \boldsymbol{L}^\top \left( \boldsymbol{L}\left( \boldsymbol{X}^\top \boldsymbol{X} \right)^{-1} \boldsymbol{L}^\top \right)^{-1} \left( \boldsymbol{L}\widehat{\boldsymbol{\beta}}^{(OLS)} - \boldsymbol{r} \right)
$$

$$
\begin{aligned}
\mathbb{V}{\rm ar} \left(\widehat{\boldsymbol{\beta}}^{(RLS)} \right) 
&= \sigma^2 \left( \mathbf{X}^\top  \mathbf{X}\right)^{-1} - \sigma^2 \left( \mathbf{X}^\top  \mathbf{X}\right)^{-1}  \mathbf{L}^\top \left( \mathbf{L}\left( \mathbf{X}^\top \mathbf{X} \right)^{-1} \mathbf{L}^\top \right)^{-1}  \mathbf{L} \left( \mathbf{X}^\top  \mathbf{X}\right)^{-1}
\end{aligned}
$$

$$
\begin{aligned} 
lnCT &= \beta_{0}+\beta_{q}lnq+ \beta_{qq}(lnq)^2+\beta_{q1}lnqlnp_1+\beta_{q2}lnqlnp_2+ \beta_{q3}lnqlnp_{3} +\beta_{1}lnp_1+\beta_{2}lnp_2+ \beta_{3}lnp_3 \\
& + \beta_{11}(lnp_{1})^2+ \beta_{22}(lnp_{2})^2+ \beta_{33}(lnp_{3})^2 + \beta_{12}lnp_{1}lnp_{2}+ \beta_{13}lnp_{1}lnp_{3}+\beta_{23}lnp_{2}lnp_{3} 
\end{aligned}
$$

ST: 

<br>
$$
\begin{aligned} 
\beta_{1} + \beta_{2} + \beta_{3} &= 1 \\
\beta_{q1} + \beta_{q2} + \beta_{q3} &= 0 \\
\beta_{11} + \beta_{12} + \beta_{13} &= 0 \\
\beta_{21} + \beta_{22} + \beta_{23} &= 0 \\
\beta_{31} + \beta_{32} + \beta_{33} &= 0 \\
\beta_{ij} = \beta_{ji}
\end{aligned}
$$

In [78]:
def restricted_OLS( X : pd.DataFrame , y : pd.Series , L : np.array, r : np.array,  intercept = True  ) -> pd.DataFrame:
    
    if not isinstance( X, pd.DataFrame ):
        raise TypeError( "X must be a pd.DataFrame." )
    
    if not isinstance( y , pd.Series ):
        raise TypeError( "y must be a pd.Series." )
    
    if not isinstance( L , np.ndarray ):
        raise TypeError( "L must be a np.ndarray." )
        
    if not isinstance( r , np.ndarray ):
        raise TypeError( "r must be a np.ndarray." )
        
    elif r.shape[ 0 ] != L.shape[ 0 ]:
        raise TypeError( "r must have the same number of rows as L" )
        
    try:
        r.shape[ 1 ] != 1
    except:
        raise ValueError( "r must be an array ( n , 1)" )
        
    if intercept == True:
        
        X[ 'Intercept' ] = 1
        # setting order of columns
        cols = X.columns.tolist()
        new_cols_orders = [ cols[ -1 ] ] + cols[ 0:-1 ]
        X = X.loc[ : , new_cols_orders ]
        
    else:
        pass
        # Data remains the same
    
    # index name
    index_names = X.columns.tolist()
    
    # From  values
    X_np = X.values
    y_np = y.values.reshape( -1 , 1 )
           
    # get beta_OLS
    beta_df = beta_OLS( X , y, intercept = False )
    beta_ols = beta_df.values.reshape( -1 , 1 )
    # get restricted beta_OLS
    beta_rls = beta_ols - np.linalg.inv( X_np.T @ X_np ) @ L.T @ np.linalg.inv( L @ np.linalg.inv( X_np.T @ X_np ) @ L.T) @ (L @ beta_ols - r )
    
#     # get standard error
#     y_fit = X_np @ beta_rls
#     resid = y_np - y_fit
#     sigma2_rls = np.sum( resid ** 2 ) / ( X_np.shape[ 0 ] - beta_rls.size - r.size )
    
#     # Get variance
#     #RA_1.dot(RA_2).dot(L)
#     D_mat = np.identity( beta_rls.size ) - ( np.linalg.inv( X_np.T @ X_np ) @ L.T @ (np.linalg.inv( L @ np.linalg.inv( X_np.T @ X_np ) @ L.T ) ) @ L ) 
#     var_cov = sigma2_rls * (D_mat @ np.linalg.inv( X_np.T @ X_np ))
    
    # get variance
    var_cov_OLS = var_OLS( X , y, intercept = False ).values
    var_cov = var_cov_OLS - var_cov_OLS  @ L.T @ np.linalg.inv( L @ np.linalg.inv(X_np.T @ X_np) @ L.T ) @ L @ np.linalg.inv( X_np.T @ X_np )
    
    # get beta standard errors
    beta_se = np.sqrt( np.diag( var_cov ) )
    
    # Calculate the test statistic for each coefficient:
    t_stat = beta_rls.ravel() / np.sqrt( np.diag( var_cov ) )
    
    # Calculate the associated p-value:
    N = X.shape[ 0 ]
    k = beta_ols.size
    rt = r.size
    p_val = 2 * stats.t.cdf( x = - np.abs( t_stat ), df =  N - k - rt )
    
    # Confidence interval
    bound = beta_se * stats.t.ppf( ( 1 + 0.95 ) / 2., N - k - rt )
    up_bd = beta_rls.ravel() + bound
    lw_bd = beta_rls.ravel() - bound
    
    
    table_data ={  'Coef.'    : beta_rls.ravel() ,
                   "Std.Err." : beta_se.ravel(),
                   "t"        : t_stat.ravel(),
                   "P>|t|"    : p_val.ravel(), 
                   "[0.025"   : lw_bd.ravel(),
                   "0.975]"   : up_bd.ravel()
                }
    
    restricted_reg_OLS = pd.DataFrame( table_data , index = index_names )
    
    return restricted_reg_OLS

In [79]:
L = np.array( ( [ 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0 ], 
              [ 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0 ], 
              [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0 ], 
              [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1 ], 
              [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1 ] ) )

r = np.array(  [  1,  0,  0,  0,  0  ]  ).reshape( -1 , 1 )

In [80]:
restricted_OLS_output = restricted_OLS( X, y, L, r ).copy()


file_to_store = open( r'../../_data/restricted_OLS_output.pkl' , "wb" )
pickle.dump( restricted_OLS_output, file_to_store )



5. Generate a new class named `RegClass`. The initial attributes of your class must be `X`, `y`, and `intercept`. `X` (covariables) must be a **pd.DataFrame** and `y` (endog) must be a **pd.Series** and `intercept` can be `True` or `False`, by default `True`. Also, you must specify the type of your class' parameters, and output and It must raise an error if the inputs do not meet the requirements.  This class should have two methods: `reg_OLS` and `restricted_OLS` (You can add more if you think it is necessary). These methods should return the same output as the previously defined functions with the same name. The method `reg_OLS` does not need an extra parameter to be executed. After executing the method `reg_OLS`, any user should access the following attributes: `beta_OLS` and `var_OLS`. They should look like `beta_OLS_output` and `var_OLS_output`. The `restricted_OLS` method needs two extra parameters before being executed, `L` and `r`. They refer to the restrictions. After running the method `restricted_OLS`, any user should access the following attributes: `beta_OLS`, `var_OLS`, `rest_beta_OLS`, and `rest_var_OLS`. These attributes refer to the beta and variance estimated using OLS and restricted OLS. They should look like  `beta_OLS_output` and `var_OLS_output`. Fix all the methods and attributes. No user should be able to change them. **Hint: Use [lecture 6](https://notebooks.githubusercontent.com/view/ipynb?browser=chrome&color_mode=auto&commit=69c80e1f2c1c268f0480a32932262201785a576c&device=unknown_device&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f616c6578616e6465727175697370652f4469706c6f6d61646f5f505543502f363963383065316632633163323638663034383061333239333232363232303137383561353736632f4c6563747572655f362f4c6563747572655f362e6970796e62&logged_in=true&nwo=alexanderquispe%2FDiplomado_PUCP&path=Lecture_6%2FLecture_6.ipynb&platform=windows&repository_id=427747212&repository_type=Repository&version=96#6.3.).**

In [3]:
import numpy as np
import pandas as pd

In [398]:
class RegClass( object ):
    
    def __init__( self, X : pd.DataFrame , y : pd.Series , intercept = True  ):
    
        if not isinstance( X, pd.DataFrame ):
            raise TypeError( "X must be a pd.DataFrame." )

        if not isinstance( y , pd.Series ):
            raise TypeError( "y must be a pd.Series." )
            
        self.X = X
        self.y = y
        self.intercept = intercept
    
        if intercept == True:

            self.X[ 'Intercept' ] = 1
            # setting order of columns
            cols = self.X.columns.tolist()
            new_cols_orders = [ cols[ -1 ] ] + cols[ 0:-1 ]
            self.X = self.X.loc[ : , new_cols_orders ]

        else:
            pass
            # Data remains the same
    
        self.X_np = self.X.values
        self.y_np = y.values.reshape( -1 , 1 )
        self.columns = self.X.columns.tolist()
    
    
    def reg_beta_OLS( self ):
        # From  values
        X_np = self.X_np
        y_np = self.y_np

        # get beta_ols
        beta_ols = np.linalg.inv( X_np.T @ X_np ) @ ( X_np.T @ y_np )

        # get columns from X
        index_names = self.columns
        # best look appereance of the output
        beta_OLS_output = pd.DataFrame( beta_ols , index = index_names , columns = [ 'Coef.' ] )
        # Make beta output an attribute
        self.beta_OLS = beta_OLS_output
        
    def reg_var_OLS( self ):
    
        # From  values
        X_np = self.X_np
        y_np = self.y_np
        
        # run functions
        self.reg_beta_OLS()
        
        # get beta_ols
        beta_OLS = self.beta_OLS.values.reshape( - 1, 1 )

        # get errors
        e = y_np - ( X_np @ beta_OLS )

        # Getting the error variance
        N = X.shape[ 0 ]
        total_parameters = X.shape[ 1 ]
        error_var = ( (e.T @ e)[ 0 ] )/( N - total_parameters )

        # Variance
        var_OLS =  error_var * np.linalg.inv( X_np.T @ X_np )

        # get columns from X to index
        index_names = self.columns
        # best look appereance of the output
        var_OLS_output = pd.DataFrame( var_OLS , index = index_names , columns = index_names )
        ## assing to an attribute
        self.var_OLS = var_OLS_output

    def reg_OLS( self ):
        
        # runing functions
        self.reg_beta_OLS()
        self.reg_var_OLS()
        X = self.X_np
        
        # var and beta
        beta_OLS = self.beta_OLS.values.reshape( -1, 1 )
        var_OLS = self.var_OLS.values
        
        # get beta standard errors
        beta_se = np.sqrt( np.diag( var_OLS ) )

        # Calculate the test statistic for each coefficient:
        t_stat = beta_OLS.ravel() / np.sqrt( np.diag( var_OLS ) )

        # Calculate the associated p-value:
        N = X.shape[ 0 ]
        k = beta.size
        p_val = 2 * stats.t.cdf( x = - np.abs( t_stat ), df =  N - k )

        # Confidence interval
        bound = beta_se * stats.t.ppf( ( 1 + 0.95 ) / 2., N - k )
        up_bd = beta.ravel() + bound
        lw_bd = beta.ravel() - bound

        table_data ={  'Coef.'    : beta_OLS.ravel() ,
                       "Std.Err." : beta_se.ravel(),
                       "t"        : t_stat.ravel(),
                       "P>|t|"    : p_val.ravel(), 
                       "[0.025"   : lw_bd.ravel(),
                       "0.975]"   : up_bd.ravel()
                    }
        
        # defining index names
        index_names = self.columns
        
        # defining a pandas dataframe 
        reg_OLS = pd.DataFrame( table_data , index = index_names )

        return reg_OLS
    
    
    def restricted_OLS( self,  L : np.array, r : np.array  ) -> pd.DataFrame:
    
        if not isinstance( L , np.ndarray ):
            raise TypeError( "L must be a np.ndarray." )

        if not isinstance( r , np.ndarray ):
            raise TypeError( "r must be a np.ndarray." )

        elif r.shape[ 0 ] != L.shape[ 0 ]:
            raise TypeError( "r must have the same number of rows as L" )

        try:
            r.shape[ 1 ] != 1
        except:
            raise ValueError( "r must be an array ( n , 1)" )
        
        # runing functions
        self.reg_beta_OLS()
        self.reg_var_OLS()
        
        # index
        index_names = self.columns
        
        # From  values
        X_np = self.X_np
        y_np = self.y_np

        # get beta_OLS
        beta_ols = self.beta_OLS.values.reshape( -1, 1 )
        # get restricted beta_OLS
        beta_rls = beta_ols - np.linalg.inv( X_np.T @ X_np ) @ L.T @ np.linalg.inv( L @ np.linalg.inv( X_np.T @ X_np ) @ L.T) @ (L @ beta_ols - r )
        rest_beta_OLS_output = pd.DataFrame( beta_rls , index = index_names , columns = [ 'Coef.' ] )
        # defining as an attribute
        self.rest_beta_OLS = rest_beta_OLS_output
        
        # get variance
        var_cov_OLS = self.var_OLS.values
        var_cov = var_cov_OLS - var_cov_OLS  @ L.T @ np.linalg.inv( L @ np.linalg.inv(X_np.T @ X_np) @ L.T ) @ L @ np.linalg.inv( X_np.T @ X_np )
        var_cov_OLS_output = pd.DataFrame( var_cov , index = index_names , columns = index_names )
        # defining as an attribute
        self.rest_var_OLS = var_cov_OLS_output

        # get beta standard errors
        beta_se = np.sqrt( np.diag( var_cov ) )

        # Calculate the test statistic for each coefficient:
        t_stat = beta_rls.ravel() / np.sqrt( np.diag( var_cov ) )

        # Calculate the associated p-value:
        N = X.shape[ 0 ]
        k = beta.size
        rt = r.size
        p_val = 2 * stats.t.cdf( x = - np.abs( t_stat ), df =  N - k - rt )

        # Confidence interval
        bound = beta_se * stats.t.ppf( ( 1 + 0.95 ) / 2., N - k - rt )
        up_bd = beta_rls.ravel() + bound
        lw_bd = beta_rls.ravel() - bound


        table_data ={  'Coef.'    : beta_rls.ravel() ,
                       "Std.Err." : beta_se.ravel(),
                       "t"        : t_stat.ravel(),
                       "P>|t|"    : p_val.ravel(), 
                       "[0.025"   : lw_bd.ravel(),
                       "0.975]"   : up_bd.ravel()
                    }

        restricted_reg_OLS = pd.DataFrame( table_data , index = index_names )

        return restricted_reg_OLS

In [370]:
file_to_store = open( r'../../_data/RegClass.pkl' , "wb" )
pickle.dump( RegClass, file_to_store )

In [371]:
A = RegClass( X, y )

In [373]:
A.reg_OLS()

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
Intercept,-76.25926,38.28004,-1.99214,0.04826,-76.74827,74.58742
lnq,-1.08043,0.37555,-2.87695,0.00463,-0.71585,0.76883
(lnq)^2,0.02649,0.00236,11.24006,0.00000,0.12638,0.13570
(lnq)(lnp1),0.13104,0.03828,3.42285,0.00081,-0.01702,0.13433
(lnq)(lnp2),0.05865,0.02555,2.29565,0.02315,-0.01036,0.09065
(lnq)(lnp3),0.04014,0.02979,1.34752,0.17994,14.65940,14.77718
lnp1,14.71829,7.08482,2.07744,0.03955,-14.89925,13.10978
lnp2,-0.89473,2.84231,-0.31479,0.75338,0.76242,11.99917
lnp3,6.38080,4.52921,1.40881,0.16106,-10.49138,7.41432
(lnp1)^2,-1.53853,0.69240,-2.22201,0.02785,-1.44217,1.29516


In [374]:
A.beta_OLS

,Coef.
Intercept,-76.25926
lnq,-1.08043
(lnq)^2,0.02649
(lnq)(lnp1),0.13104
(lnq)(lnp2),0.05865
(lnq)(lnp3),0.04014
lnp1,14.71829
lnp2,-0.89473
lnp3,6.38080
(lnp1)^2,-1.53853


In [376]:
A.var_OLS

,Intercept,lnq,(lnq)^2,(lnq)(lnp1),(lnq)(lnp2),(lnq)(lnp3),lnp1,lnp2,lnp3,(lnp1)^2,(lnp2)^2,(lnp3)^2,(lnp1)(lnp2),(lnp1)(lnp3),(lnp2)(lnp3)
Intercept,1465.36113,0.19254,-0.01170,-0.02668,0.01595,0.03954,-262.93257,-8.39268,-128.19205,23.76171,-0.53389,7.08128,1.98051,21.80488,0.39487
lnq,0.19254,0.14103,0.00031,-0.01346,0.00016,-0.00609,-0.18622,0.14832,-0.09071,0.03064,0.00076,-0.00180,-0.03009,0.03509,-0.00714
(lnq)^2,-0.01170,0.00031,0.00001,-0.00004,-0.00001,-0.00001,0.00149,0.00105,0.00086,-0.00009,0.00003,-0.00012,-0.00020,-0.00003,-0.00010
(lnq)(lnp1),-0.02668,-0.01346,-0.00004,0.00147,-0.00017,0.00035,0.02156,-0.01513,0.00543,-0.00392,0.00004,0.00057,0.00370,-0.00228,-0.00017
(lnq)(lnp2),0.01595,0.00016,-0.00001,-0.00017,0.00065,-0.00019,-0.00829,0.00331,0.00803,0.00117,-0.00034,-0.00021,-0.00034,-0.00029,-0.00275
(lnq)(lnp3),0.03954,-0.00609,-0.00001,0.00035,-0.00019,0.00089,-0.00034,-0.00885,0.00027,0.00042,-0.00007,-0.00013,0.00020,-0.00307,0.00442
lnp1,-262.93257,-0.18622,0.00149,0.02156,-0.00829,-0.00034,50.19472,-1.25353,19.09954,-4.77797,0.14099,-0.84031,0.07683,-3.57632,0.26431
lnp2,-8.39268,0.14832,0.00105,-0.01513,0.00331,-0.00885,-1.25353,8.07874,-0.21487,0.32329,-0.11729,-0.06629,-1.37010,0.39896,-0.73478
lnp3,-128.19205,-0.09071,0.00086,0.00543,0.00803,0.00027,19.09954,-0.21487,20.51371,-1.41043,0.04733,-1.52610,0.07246,-3.11111,-0.14518
(lnp1)^2,23.76171,0.03064,-0.00009,-0.00392,0.00117,0.00042,-4.77797,0.32329,-1.41043,0.47942,-0.01232,0.04541,-0.05365,0.28152,-0.02662


In [378]:
A.restricted_OLS( L , r )

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
Intercept,-7.11120,2.75587,-2.58039,0.01091,-12.56038,-1.66202
lnq,0.46137,0.11972,3.85368,0.00018,0.22464,0.69809
(lnq)^2,0.02998,0.00212,14.13476,0.00000,0.02578,0.03417
(lnq)(lnp1),-0.00435,0.02288,-0.19031,0.84935,-0.04960,0.04089
(lnq)(lnp2),0.03678,0.02420,1.51967,0.13088,-0.01108,0.08463
(lnq)(lnp3),-0.03242,0.02142,-1.51378,0.13237,-0.07478,0.00993
lnp1,0.21557,1.01521,0.21234,0.83216,-1.79180,2.22294
lnp2,0.40292,0.55416,0.72709,0.46841,-0.69282,1.49867
lnp3,0.38151,0.53132,0.71805,0.47394,-0.66906,1.43208
(lnp1)^2,-0.00871,0.14096,-0.06181,0.95081,-0.28743,0.27001


In [379]:
A.rest_var_OLS

,Intercept,lnq,(lnq)^2,(lnq)(lnp1),(lnq)(lnp2),(lnq)(lnp3),lnp1,lnp2,lnp3,(lnp1)^2,(lnp2)^2,(lnp3)^2,(lnp1)(lnp2),(lnp1)(lnp3),(lnp2)(lnp3)
Intercept,7.59480,-0.08606,0.00009,0.01403,-0.01978,0.00575,-2.73017,1.44411,1.28606,0.35488,-0.08228,-0.73581,-0.50421,0.14933,0.58649
lnq,-0.08606,0.01433,-0.00003,-0.00261,0.00203,0.00058,0.01091,-0.01319,0.00228,0.00143,-0.00261,-0.00118,0.00000,-0.00143,0.00261
(lnq)^2,0.00009,-0.00003,0.00000,-0.00001,-0.00000,0.00001,-0.00002,0.00007,-0.00005,0.00001,0.00003,-0.00004,-0.00004,0.00003,0.00001
(lnq)(lnp1),0.01403,-0.00261,-0.00001,0.00052,-0.00033,-0.00020,-0.00123,0.00152,-0.00029,-0.00043,0.00050,0.00051,0.00022,0.00021,-0.00072
(lnq)(lnp2),-0.01978,0.00203,-0.00000,-0.00033,0.00059,-0.00026,0.00456,-0.00548,0.00091,-0.00030,0.00005,0.00041,0.00033,-0.00003,-0.00038
(lnq)(lnp3),0.00575,0.00058,0.00001,-0.00020,-0.00026,0.00046,-0.00333,0.00396,-0.00062,0.00073,-0.00055,-0.00093,-0.00055,-0.00018,0.00110
lnp1,-2.73017,0.01091,-0.00002,-0.00123,0.00456,-0.00333,1.03064,-0.52772,-0.50292,-0.14003,0.04081,0.27716,0.18819,-0.04816,-0.22900
lnp2,1.44411,-0.01319,0.00007,0.00152,-0.00548,0.00396,-0.52772,0.30710,0.22062,0.06985,-0.00228,-0.15324,-0.11041,0.04056,0.11268
lnp3,1.28606,0.00228,-0.00005,-0.00029,0.00091,-0.00062,-0.50292,0.22062,0.28230,0.07019,-0.03853,-0.12392,-0.07779,0.00760,0.11632
(lnp1)^2,0.35488,0.00143,0.00001,-0.00043,-0.00030,0.00073,-0.14003,0.06985,0.07019,0.01987,-0.00739,-0.03755,-0.02501,0.00514,0.03240


In [380]:
A.rest_beta_OLS

,Coef.
Intercept,-7.11120
lnq,0.46137
(lnq)^2,0.02998
(lnq)(lnp1),-0.00435
(lnq)(lnp2),0.03678
(lnq)(lnp3),-0.03242
lnp1,0.21557
lnp2,0.40292
lnp3,0.38151
(lnp1)^2,-0.00871
